In [10]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network
import scipy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pandas as pd
import scipy.io
import collections
import sys
import scipy.optimize
import os
import os.path
from math import comb
from game import Game, GameResult

# sys.path.append('../gameutils')
from utils import *

save_path = os.getcwd()

calculate_polarization = calculate_polarization1

    # Fixed initial condition + memeory = 50

## Mathmatic Functions

### 1. Import Network

In [11]:
# # #################### Import Sythetic Network Data
df = pd.read_csv (save_path+'/data/5Adj_matrix.csv', header=None)
G = np.array(df[df.columns[:]])
# # # print(G)
df1 = pd.read_csv(save_path+'/data/5innate_op.csv', header = None)
s = np.array(df1[df1.columns[:]])

# print(s)
# print(G.shape)

# # Set n according to the data
n = len(s[:])

In [12]:
################## Process the Network Data
L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
m = num_edges(L, n)                    # call the function to calculate the number of edges
# what the twitter graph looks like
nxG = nx.from_numpy_array(G)
#plt.figure(figsize=(20, 20))
# nx.draw(nxG).
columnsum_ij = np.sum(A, axis=0)
print(columnsum_ij)
# s = [0.5]*n
# s = np.reshape(s, (n, 1))
print(s)

[1. 1. 1. 1. 1.]
[[0.231]
 [0.887]
 [0.641]
 [0.218]
 [0.162]]


In [13]:
# what the twitter graph looks like
s_use = s.flatten()   # Convert array to a list for later operation
s_use = s_use.tolist()
new_s = [i * 30 for i in s_use]
df = pd.DataFrame(new_s, columns=['Opinion']) #create a dataframe with index at column 1, opinion at column 2
#_______________________________________________________

nxG = nx.from_numpy_array(G)
# nxG = nx.relabel_nodes(nxG, mapping)
plt.figure(figsize=(20, 20))


def node_edge(G, n):
    edges =[]
    for v in range(n):
        a = np.array(np.nonzero(G[v])[0])
        edge = len(a)
#         print(edge)
        edges.append(edge)

    return edges

node_edges = node_edge(G, n)
# print(node_edges)

node_sizes =[]
for i in node_edges:
    node_size = 1/i*20000
    node_sizes.append(node_size)

min_equi = []
max_equi = []
# Create node color
color_map = []
for node in nxG:
    if node in min_equi:
        color_map.append('orange')
    elif node in max_equi:
        color_map.append('red')
    else:
        color_map.append('grey')
# print(node_sizes)
#-_______________________________________
# what the twitter graph looks like
nxG = nx.from_numpy_array(G)
plt.figure(figsize=(20, 20))

# # Fix seed - fix network shape
# my_pos = nx.spring_layout(nxG, seed = 2)
# nx.draw(nxG, pos= my_pos, with_labels= True, node_color=df['Opinion'].astype(int),cmap=plt.cm.Blues, node_size= node_sizes, edge_color='black', width=0.8, font_color='black',font_size=26, font_weight='bold', alpha=0.8)
# #nx.draw(nxG, pos = my_pos, with_labels=False, node_color=color_map, node_size= node_sizes, edge_color='grey', width=0.5, font_color='white',font_size=9, font_weight='bold')
# sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(vmin = 0, vmax=1))
# cbar = plt.colorbar(sm, shrink = 0.5)
# tick_font_size = 24
# cbar.ax.tick_params(labelsize=tick_font_size)
# plt.show()


<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

### 2. Equilibrium & Polarization 
$$P(z) = z ^T * z $$


## 5. Innate Op and Game

# Fictitious Play Start !


In [14]:
# Calculate network polarization
calculate_polarization(s, n, A, L)

Innate_polarization:	0.41022891110361115
Equi_polarization:	0.04411213470472514
Difference:		-0.366116776398886


In [15]:
# Game Parameters
game_rounds = 3
memory = 10
k = 2
experiment = 1

In [16]:
game = Game(s, A, L, k)
result = game.run(game_rounds, memory)

Nodes, opinions
[0, 3] (1, 1)
Nodes, opinions
[1, 4] (0.5, 0.5)
min_history [((1, 4), (0.5, 0.5))]
--------------------
Game 1
----------
min_history [((1, 4), (0.5, 0.5))]
max_history [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
fla_min_fre [1.]
Max_por 0.033797021050447185
Max_por 0.033797021050447185
Max_por 0.033797021050447185
all_por [0.    0.    0.    0.    0.028 0.022 0.022 0.034 0.016 0.019 0.024 0.016
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.019 0.028 0.02  0.018 0.    0.    0.    0.
 0.    0.    0.    0.   ]
column - best action: 7
Maximizer found its target 2 agent: [0, 2] op: (1, 1)
fre_max at spot 0.5
M
[[ 0.159 -0.041  0.159 -0.038 -0.241]]
M
[[ 0.139 -0.061  0.068  0.079 -0.225]]
champion:  [1, 4] [0.834 0.877] 0.008232769862068768
Minimizer finds its target agents: [1, 4]
fla_min_fre: [0.5 0.5]
Not Reached Nash Equilibrium at Equi_Min = 0.008232769862068768 and Equi_Max = 0.03

In [17]:
def save(result: GameResult, k, experiment):
    pd.DataFrame(result.payoff_matrix).to_csv(
        f'results/Payoff-Matrix-k-{k}-experiment-{experiment}.csv')

    # Save the original standard output
    original_stdout = sys.stdout

    with open(f'results/Result-k-{k}-experiment-{experiment}.txt', "w") as f:
        # Change the standard output to the file we created.
        sys.stdout = f

        print('Initial Condition -(agent, opinion, pol)')
        # print(f'Innate op {s}')
        # print(f'Adjacency matrix {G}')
        # print('Selected Nodeset, k_Opinions, Steady-state polarization')
        print(f'Max:\t{result.first_max}')
        print(f'Min:\t{result.first_min}')

        print('_____________________')
        print(f'Max Pol:\t{result.equi_max}')
        print(f'Min Pol:\t{result.equi_min}')

        # MAXimizer's distribution of LAST 100 iteration
        print('Max_distribution_last_100')
        max_l100_fre = result.max_history_last_100/100
        print(max_l100_fre[np.nonzero(max_l100_fre)])
        # print for small network
        # print(max_history_last_100)
        # # Print for Large Network
        print(np.nonzero(max_l100_fre))

        columns = np.nonzero(max_l100_fre)
        columns = list(columns[0])
        for column in columns:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        print('Max_distribution_all')
        max_fre = result.max_history/result.game_rounds
        print(max_fre[np.nonzero(max_fre)])
        print([np.nonzero(max_fre)])

        # TODO confirm isn't this block the same as the one above
        columns_all = np.nonzero(max_l100_fre)
        columns_all = list(columns_all[0])
        for column in columns_all:
            (k_nodes, opinions) = game.map_action(column)
            print(f'Max Nodes: {k_nodes}\tOpinion: {opinions}')

        # MINimizer's Strategy in the last 100 round
        counter = collections.Counter(result.min_touched_last_100)
        # frequency of all min options in order
        fla_min_fre = np.array(list(counter.values()))/(100)
        print('Min_distribution_last_100')
        print(fla_min_fre)
        print(counter)
        # print(min_touched_last_100)

        # a dictionary include {'min_option': count of this choice}
        counter_1 = collections.Counter(result.min_touched_all)
        # frequency of all min options in order
        fla_min_fre_1 = np.array(list(counter_1.values()))/result.game_rounds
        print('Min_distribution_all')
        print(fla_min_fre_1)
        print(counter_1)
        np.set_printoptions(precision=3)

        # a dictionary include {'min_option': count of this choice}
        counter_a = collections.Counter(result.min_history)
        print(counter_a)

        print(f'min_recent_{memory}_touched')
        print(result.min_touched)
        print(f'max_recent_{memory}_touched')
        print(result.max_touched)

        # Reset the standard output to its original value
        sys.stdout = original_stdout

In [18]:
save(result, k, experiment)